In [32]:
import pandas as pd
from patsy import dmatrix
import numpy as np
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy as sp
import warnings
warnings.filterwarnings('always')
plt.rc("font", family="Malgun Gothic")
mpl.rcParams['axes.unicode_minus'] = False

In [2]:
# 행렬의 조건수가 높으면 회귀분석이 정상적으로 안됨 > 스케일링의 필요성

In [9]:
A = np.eye(4)
A

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

In [10]:
b = np.ones(4)
b

array([1., 1., 1., 1.])

In [11]:
x = sp.linalg.solve(A, b)
x

array([1., 1., 1., 1.])

In [13]:
x_error = sp.linalg.solve(A + 0.0001 * np.eye(4), b)
x_error

array([0.99990001, 0.99990001, 0.99990001, 0.99990001])

In [14]:
# 조건수가 작으면 위와 같이 약간의 오차에도 아웃풋이 큰 차이가 없음.
# 그러나 조건수가 크면(예시 힐베르트 행렬) 아웃풋에 영향이 커서 회귀분석 불가

In [16]:
np.linalg.cond(A) # 조건수

1.0

In [17]:
A2 = sp.linalg.hilbert(4)
A2

array([[1.        , 0.5       , 0.33333333, 0.25      ],
       [0.5       , 0.33333333, 0.25      , 0.2       ],
       [0.33333333, 0.25      , 0.2       , 0.16666667],
       [0.25      , 0.2       , 0.16666667, 0.14285714]])

In [19]:
np.linalg.cond(A2)  # 조건수가 큼

15513.738738929038

In [20]:
x = sp.linalg.solve(A2, b)
x

array([  -4.,   60., -180.,  140.])

In [22]:
x_error = sp.linalg.solve(A2 + 0.0001 * np.eye(4), b)
x_error

array([ -0.58897672,  21.1225671 , -85.75912499,  78.45650825])

In [23]:
#약간으 오차에 큰 영향이 생김 > 스케일링 필요

In [26]:
# 보스턴 데이터 각 변수들의 표준편차  > 편차가 심함
from sklearn.datasets import load_boston
boston = load_boston()
df = pd.DataFrame(boston.data, columns=boston.feature_names)
df.describe().loc['std']

CRIM         8.601545
ZN          23.322453
INDUS        6.860353
CHAS         0.253994
NOX          0.115878
RM           0.702617
AGE         28.148861
DIS          2.105710
RAD          8.707259
TAX        168.537116
PTRATIO      2.164946
B           91.294864
LSTAT        7.141062
Name: std, dtype: float64

In [27]:
dfy = pd.DataFrame(boston.target, columns=['MEDV'])

In [40]:
df2 = df.copy()
df2['TAX'] += 1e13
df_xy = pd.concat([df2, dfy], axis=1)

In [41]:
df_xy[100:102]

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
100,0.14866,0.0,8.56,0.0,0.52,6.727,79.9,2.7778,5.0,1.000000e+13,20.9,394.76,9.42,27.5
101,0.11432,0.0,8.56,0.0,0.52,6.781,71.3,2.8561,5.0,1.000000e+13,20.9,395.58,7.67,26.5


In [42]:
import statsmodels.api as sm
model = sm.OLS.from_formula('MEDV ~' + '+'.join(boston.feature_names), data=df_xy)
result = model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                   MEDV   R-squared:                       0.735
Model:                            OLS   Adj. R-squared:                  0.729
Method:                 Least Squares   F-statistic:                     113.9
Date:                Sun, 28 Nov 2021   Prob (F-statistic):          1.21e-133
Time:                        17:14:46   Log-Likelihood:                -1504.3
No. Observations:                 506   AIC:                             3035.
Df Residuals:                     493   BIC:                             3089.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   1.536e-10   2.93e-11      5.250      0.0